In [1]:
import os
import pandas as pd
import numpy as np
import datetime
import math
import io
import boto3
import json
import dask.dataframe as dd
from pyspark.sql import functions
import awswrangler as wr
from sqlalchemy import create_engine
from sqlalchemy.pool import NullPool
from concurrent.futures import ThreadPoolExecutor
from pandas.tseries.offsets import MonthEnd, MonthBegin

#### Formatação de datas pra pegar os parquets do mês atual e anterior da tabela payments

In [2]:
today = datetime.date.today()
dateLastMonth = today.replace(day=1) - datetime.timedelta(days=1)
month = today.strftime("%m")
monthYear = today.strftime("%Y%m")
yearMonth = today.strftime("%Y")
lastMonth = dateLastMonth.strftime("%m")
yearLastMonth = dateLastMonth.strftime("%Y")
lastMonthYear = dateLastMonth.strftime("%Y%m")

In [3]:
#teste
month = "10"
lastMonth = "09"
yearMonth = "2021"

In [4]:
dtMonthPayments = dd.read_parquet('s3://dlr-dev-bucket-refinedzone/pic/datamart/payments/year='+yearMonth+'/month='+month+'/').compute()

In [5]:
dtLastMonthPayments = dd.read_parquet('s3://dlr-dev-bucket-refinedzone/pic/datamart/payments/year='+yearMonth+'/month='+lastMonth+'/').compute()

In [6]:
payments = pd.concat([dtMonthPayments, dtLastMonthPayments], ignore_index=True)

In [7]:
del dtLastMonthPayments, dtMonthPayments

In [8]:
AssignmentsOptimizationRestrictive = dd.read_parquet('s3://dlr-dev-bucket-rawzone/pic/datascience/assignmentsoptimizationrestrictive/AssignmentsOptimizationRestrictive.parquet').compute()


In [9]:
AssignmentsOptimizationRestrictiveSegments = pd.read_parquet('s3://dlr-dev-bucket-rawzone/pic/datascience/assignmentsoptimizationrestrictivesegments/AssignmentsOptimizationRestrictiveSegments.parquet')

In [10]:
AssignmentsOptimizationRestrictiveSegments.head()

,SegmentID,Segment,CreationDate,isEnabled
0,1,Agibank,2021-09-28 09:14:05.910,True
1,2,Aymore,2021-09-28 09:14:05.910,True
2,3,BancoPan,2021-09-28 09:14:05.910,True
3,4,Bradesco-Banco,2021-09-28 09:14:05.910,True
4,5,Bradesco-BBF,2021-09-28 09:14:05.910,True


In [11]:
AssignmentsOptimizationRestrictivePortfolios = pd.read_parquet('s3://dlr-dev-bucket-rawzone/pic/datascience/assignmentsoptimizationrestrictiveportfolios/AssignmentsOptimizationRestrictivePortfolios.parquet')

In [12]:
AssignmentsOptimizationRestrictivePortfolios.tail()

,ID,SegmentID,PortfolioID,CreationDate,isEnabled,ResponsableName
57,58,46,114,2021-12-27 15:50:22.033,True,Pedro Granero
58,59,47,115,2021-12-27 15:50:22.033,True,Pedro Granero
59,60,48,116,2021-12-27 15:50:22.033,True,Pedro Granero
60,61,49,117,2021-12-27 15:50:22.033,True,Pedro Granero
61,62,50,118,2021-12-27 15:50:22.033,True,Pedro Granero


In [13]:
businessUnits = dd.read_parquet('s3://dlr-dev-bucket-rawzone/pic/ftcrm/businessunits/*.parquet').compute()

In [14]:
businessUnits = businessUnits.rename(columns={"businessunitid":"AgencyID"})

### Definição das datas necessárias pra análise

In [15]:
today = datetime.date.today()
reference_date = today - MonthBegin() # mÊs atual
optimization_month = reference_date + MonthBegin() # próximo mês

In [16]:
mes_atual = optimization_month - MonthBegin()
mes_anterior = mes_atual - MonthBegin()
cuts_age = [0, 5, 10, math.inf]

In [17]:
mes_atual = pd.to_datetime(mes_atual, format="%Y-%m-%d")
mes_anterior = pd.to_datetime(mes_anterior, format="%Y-%m-%d")


In [18]:
#teste
mes_anterior = pd.to_datetime('2021-09-01', format="%Y-%m-%d")
mes_atual = pd.to_datetime('2021-10-01', format="%Y-%m-%d")

### Carrega o nome das agências

In [19]:
agency = businessUnits[["AgencyID", "name", "alias"]]

In [20]:
agency = agency.rename(columns={'AgencyID': 'agency_id'})

In [21]:
agency.head()

,agency_id,name,alias
0,1,Hoepers,HOE01
1,5,ASS 001,1
2,6,CONTRATO,9
3,7,QUALITY,18
4,8,INTERVAL,74


###  Listando portfolio ativos e com  AccountOwnerID = 672

In [22]:
portfolio_ids__enabled = AssignmentsOptimizationRestrictivePortfolios[AssignmentsOptimizationRestrictivePortfolios.isEnabled == 1][["PortfolioID"]]

In [23]:
portfolio_ids__enabled.sort_values(by=['PortfolioID'], inplace=True)

In [24]:
portfolio_ids__enabled.head()

,PortfolioID
27,46
33,47
20,48
21,49
34,52


In [25]:
portifolios = portfolio_ids__enabled["PortfolioID"].tolist()

### Seleciona dividas que não estão com acordos ativos 
#### As dividas que possuem acordos ativos são marcadas com bindingid not NULL, bindingid é usado para direcionar dívidas para as diferentes agencias

In [26]:
debtcontacts_bindingid_null = dd.read_parquet('s3://dlr-dev-bucket-refinedzone/pic/datamart/debtcontacts/part*.parquet', columns=["DebtID","DebtStatus","BindingID", "AccountOwnerID"]).compute()

In [27]:
debt_bindingid_null = debtcontacts_bindingid_null[(debtcontacts_bindingid_null.BindingID.isnull()) & (debtcontacts_bindingid_null.AccountOwnerID == 672)]

In [28]:
debt_bindingid_null = debt_bindingid_null.rename(columns={'DebtID': 'debt_id', 'DebtStatus': 'status'})

In [29]:
debt_bindingid_null.head()

,debt_id,status,BindingID,AccountOwnerID
182145,14070125.0,NaN,NaN,672.0


### Seleciona dividas ajuizadas 
##### Dívidas ajuizadas não devem ser cobradas

In [30]:
debtcontacts = dd.read_parquet('s3://dlr-dev-bucket-refinedzone/pic/datamart/debtcontacts/part*.parquet', columns=["Assets","Liabilities", "CustomerID", "AccountOwnerID", "DebtStatus"]).compute()

In [31]:
customer_ajuizado = debtcontacts[(debtcontacts.AccountOwnerID == 672) & (debtcontacts.DebtStatus == 7000) & ((debtcontacts.Assets == 1) | (debtcontacts.Liabilities == 1))]

In [32]:
del debtcontacts

In [33]:
customer_ajuizado = customer_ajuizado.rename(columns={'Assets': 'assets', 'Liabilities': 'liabilities',"CustomerID": "customer_id"})

In [34]:
customer_ajuizado = customer_ajuizado.groupby("customer_id", as_index = False)[["assets", "liabilities" ]].max()

### Verifica as agências que estão habilitadas receber dívidas

In [35]:
AssignmentsOptimizationRestrictive.head()

,ID,SegmentID,AgencyID,ReferenceBalance,AvailableBalance,DistributionDate,isEnabled,MinValue,MaxValue,EnforcedFlag
0,1,1,185,1.206874e+08,18036391.62,2021-10-01,True,NaN,NaN,False
1,2,1,519,0.000000e+00,0.00,2021-10-01,True,NaN,NaN,False
2,3,1,571,2.842417e+07,7398771.38,2021-10-01,True,NaN,NaN,False
3,4,1,640,1.369426e+07,13694257.56,2021-10-01,False,NaN,NaN,False
4,5,1,652,1.336905e+04,13369.05,2021-10-01,True,NaN,NaN,False


In [36]:
segment_restriction = pd.merge(AssignmentsOptimizationRestrictiveSegments, AssignmentsOptimizationRestrictive, on="SegmentID")

In [37]:
#teste
data = '2021-11-01 00:00:00'

In [38]:
segment_restriction = segment_restriction[segment_restriction.DistributionDate == data][["SegmentID","Segment","AgencyID","ReferenceBalance","AvailableBalance","isEnabled_y","EnforcedFlag"]]

In [39]:
segment_restriction = segment_restriction.rename(columns={'SegmentID':'segment_id', 'Segment':'segment', 'AgencyID':'agency_id', 'ReferenceBalance':'reference_balance','AvailableBalance':'available_balance','isEnabled_y':'is_enabled','EnforcedFlag':'enforced_flag'})

In [40]:
segment_restriction.sort_values(by=['segment_id'], inplace=True)

In [41]:
segment_restriction.head()

,segment_id,segment,agency_id,reference_balance,available_balance,is_enabled,enforced_flag
15,1,Agibank,185,1.239133e+08,18170827.43,True,False
29,1,Agibank,809,3.224902e+05,322490.21,False,False
28,1,Agibank,796,NaN,NaN,False,False
27,1,Agibank,794,1.664443e+04,16644.43,False,False
26,1,Agibank,699,1.420509e+08,84828041.06,True,False


### Seleciona os segmentos disponíveis

In [42]:
segment_portfolio = AssignmentsOptimizationRestrictivePortfolios[AssignmentsOptimizationRestrictivePortfolios.isEnabled == 1][["SegmentID", "PortfolioID"]]

In [43]:
segment_portfolio = segment_portfolio.rename(columns={'SegmentID': 'segment_id', 'PortfolioID':'portfolio_id'})

In [44]:
segment_portfolio.head()

,segment_id,portfolio_id
0,1,68
1,2,86
2,3,55
3,3,70
4,4,57


## new_assignments_template 
### Seleciona as distribuições mais recentes

### Selecionar as distribuições ativas que estarão com AssignmentStatus != 95005

In [45]:
#assignments_new = dd.read_parquet('s3://dlr-dev-bucket-rawzone/pic/datamart/assignment-debts-analytical/*.parquet', columns = ["AssignmentDate", "DebtID", "AgencyID", "AssignmentID", "DebtStatus", "DebtStatusID"]).compute()

In [52]:
assignments_new = dd.read_parquet('s3://dlr-dev-bucket-rawzone/pic/datamart/assignment-debts-analytical/outubro/*.parquet', columns = ["AssignmentDate", "DebtID", "AgencyID", "AssignmentID", "DebtStatus", "DebtStatusID"]).compute() 

In [53]:
#teste
assignments_new2 = dd.read_parquet('s3://dlr-dev-bucket-rawzone/pic/datamart/assignment-debts-analytical/setembro/*.parquet', columns = ["AssignmentDate", "DebtID", "AgencyID", "AssignmentID", "DebtStatus", "DebtStatusID"]).compute() 

In [54]:
assignments_new = pd.concat([assignments_new, assignments_new2])

In [55]:
len(assignments_new)

37791004

In [56]:
assignments_new = assignments_new.drop_duplicates()

In [57]:
len(assignments_new)

19908957

In [58]:
assignments_new["AssignmentDate"] = pd.to_datetime(assignments_new["AssignmentDate"])

In [59]:
#teste
reference_date = pd.to_datetime('2021-10-01', format="%Y-%m-%d")
reference_date

Timestamp('2021-10-01 00:00:00')

In [60]:
assignmentDate = assignments_new[(assignments_new.AssignmentDate <= reference_date) & (assignments_new.DebtStatus == 'Active')]

In [61]:
debtcontacts_new = pd.read_parquet('s3://dlr-dev-bucket-refinedzone/pic/datamart/debtcontacts/', columns = ["PortfolioID", "CustomerID", "BindingID", "DebtID", "SettlementDate", "RecallDate", "SellDate", "DebtStatus"])

In [62]:
debtcontacts_new_portifolios = debtcontacts_new[debtcontacts_new.PortfolioID.isin(portifolios)]

In [63]:
max_assignment = pd.merge(assignmentDate, debtcontacts_new_portifolios, on="DebtID")

In [64]:
max_assignment.head()

,AssignmentDate,DebtID,AgencyID,AssignmentID,DebtStatus_x,DebtStatusID,PortfolioID,CustomerID,BindingID,SettlementDate,RecallDate,SellDate,DebtStatus_y
0,2021-07-30,35379265,680,367414416,Active,7000.0,78.0,49986723.0,33172696.0,NaT,NaT,None,7000.0
1,2020-06-15,12992389,756,316449960,Active,7000.0,80.0,25357717.0,13066713.0,NaT,NaT,None,7000.0
2,2021-02-27,20801440,684,350266498,Active,7000.0,53.0,41134400.0,20224573.0,NaT,NaT,None,7000.0
3,2021-04-29,25313211,681,355441292,Active,7000.0,58.0,46408643.0,24788820.0,NaT,NaT,None,7000.0
4,2020-10-03,30790319,756,334264819,Active,7000.0,66.0,51097938.0,27306831.0,NaT,NaT,None,7000.0


In [65]:
del debtcontacts_new_portifolios, assignmentDate

In [66]:
max_assignment = max_assignment.groupby("DebtID")[["AssignmentID"]].min()

#### Verifica se o debtos estão abertos


In [67]:
ref_date = reference_date

In [68]:
debtcontacts_new["SellDate"] = pd.to_datetime(debtcontacts_new["SellDate"])

In [69]:
debts = debtcontacts_new[(debtcontacts_new.SettlementDate.isnull() | debtcontacts_new.RecallDate.isnull() | debtcontacts_new.SellDate.isnull()) | ((debtcontacts_new.SettlementDate > ref_date) & (~debtcontacts_new.SettlementDate.isnull())) | ((~debtcontacts_new.RecallDate.isnull()) & (debtcontacts_new.RecallDate > ref_date)) | ((debtcontacts_new.SellDate > ref_date) & (~debtcontacts_new.SellDate.isnull()))]

In [70]:
debts.head()

,PortfolioID,CustomerID,BindingID,DebtID,SettlementDate,RecallDate,SellDate,DebtStatus
0,79.0,19551642.0,12322098.0,12230411.0,NaT,NaT,NaT,7000.0
1,79.0,19540108.0,12299682.0,12230412.0,NaT,NaT,NaT,7000.0
2,79.0,19521919.0,12262747.0,12230413.0,NaT,NaT,NaT,7000.0
3,79.0,7267952.0,12232688.0,12230414.0,NaT,NaT,NaT,7000.0
4,79.0,19528076.0,12275568.0,12230416.0,NaT,NaT,NaT,7000.0


In [72]:
debts['DebtStatus'] = debts['DebtStatus'].fillna(0)

In [73]:
debts['DebtStatus'] = debts['DebtStatus'].astype('int')

#### Pega apenas as dívidas disponíveis para distribuição DebtStatus = 7000

In [75]:
debts = debts[debts.DebtStatus == 7000]

#### Remove os escritórios que são digitais, esses podem estar a dívida concomitantemente em mais de um escritório

In [76]:
units = businessUnits[businessUnits.businessunittypeid.isin([4,8])]

#### Remove algumas agências

In [77]:
ass = assignments_new[~assignments_new.AgencyID.isin(["716","175","756","820"])]

In [78]:
new_assignments_template = pd.merge(pd.merge(pd.merge(ass,max_assignment,on='AssignmentID'),debts,on='DebtID'), units, on="AgencyID")

In [80]:
new_assignments_template[new_assignments_template.DebtID == 35379265 ]

,AssignmentDate,DebtID,AgencyID,AssignmentID,DebtStatus_x,DebtStatusID,PortfolioID,CustomerID,BindingID,SettlementDate,...,contactid,parentbusinessunitid,creationdate,creationusersid,dataversion,businessunittypeid,enabled,businessunitcode,email,cancellationdate
0,2021-07-30,35379265,680,367414416,Active,7000.0,78.0,49986723.0,33172696.0,NaT,...,35123687.0,NaN,2017-12-28 12:04:00,0,b'\x00\x00\x00\x06a\xd8\xc5\xdb',4,True,None,None,None


In [81]:
new_assignments_template[new_assignments_template.duplicated()]

,AssignmentDate,DebtID,AgencyID,AssignmentID,DebtStatus_x,DebtStatusID,PortfolioID,CustomerID,BindingID,SettlementDate,...,contactid,parentbusinessunitid,creationdate,creationusersid,dataversion,businessunittypeid,enabled,businessunitcode,email,cancellationdate


In [82]:
del ass, max_assignment, debts, debtcontacts_new, assignments_new, businessUnits

In [83]:
new_assignments_template= new_assignments_template[["AssignmentDate", "PortfolioID", "DebtID", "CustomerID", "AgencyID", "BindingID"]]

In [84]:
new_assignments_template.columns = ['assignment_time', 'portfolio_id','debt_id','customer_id','agency_id','binding_id']

In [85]:
new_assignments_template.sort_values(by=['portfolio_id', 'debt_id'], inplace=True)

In [86]:
new_assignments_template.head()

,assignment_time,portfolio_id,debt_id,customer_id,agency_id,binding_id
818080,2021-02-27,46.0,14070137,26969668.0,481,14154373.0
1621317,2021-06-02,46.0,14070139,26965502.0,640,14369179.0
649186,2021-05-28,46.0,14070141,26988066.0,691,14152418.0
1672105,2021-06-29,46.0,14070144,26965416.0,635,14163849.0
838719,2021-08-31,46.0,14070157,26976693.0,481,14167591.0


### process_portfolio

### Verifica para quais agências as dívidas foram distribuidas no mês atual 

In [87]:
#teste
assingments_process_portfolio = dd.read_parquet('s3://dlr-dev-bucket-rawzone/pic/datamart/assignment-debts-analytical/outubro/*.parquet', columns = ["RefDate","DebtID", "PortfolioID", "CustomerID", "AgencyID", "AssignmentDate"]).compute() 

In [88]:
#teste
assingments_process_portfolio2 = dd.read_parquet('s3://dlr-dev-bucket-rawzone/pic/datamart/assignment-debts-analytical/setembro/*.parquet', columns = ["RefDate","DebtID", "PortfolioID", "CustomerID", "AgencyID", "AssignmentDate"]).compute() 

In [89]:
len(assingments_process_portfolio)

19256634

In [90]:
len(assingments_process_portfolio2)

18534370

In [91]:
assingments_process_portfolio = pd.concat([assingments_process_portfolio, assingments_process_portfolio2])

In [92]:
assingments_process_portfolio = assingments_process_portfolio.drop_duplicates()

In [93]:
assingments_process_portfolio["AssignmentDate"] = pd.to_datetime(assingments_process_portfolio["AssignmentDate"])

In [94]:
assingments_process_portfolio["RefDate"] = pd.to_datetime(assingments_process_portfolio["RefDate"], format="%Y%m")

In [95]:
# temp_start = assingments_process_portfolio["AssignmentDate"].max() - MonthBegin(1)

In [96]:
# ref_date_month = pd.to_datetime(temp_start, format="%Y-%m-%d") + pd.offsets.Day(10)

In [97]:
# ref_date_month

In [98]:
# assingments_process_portfolio.dtypes

In [99]:
# assingments_process_portfolio["months_in_agency"] = np.floor((ref_date_month - assingments_process_portfolio["AssignmentDate"])/np.timedelta64(1, 'M'))

In [100]:
assingments_process_portfolio.head()

,RefDate,DebtID,PortfolioID,CustomerID,AgencyID,AssignmentDate
0,2021-10-01,30109227,66,26954370,685,2020-02-01
1,2021-10-01,33897309,75,54904329,794,2021-07-30
2,2021-10-01,24913704,57,25676425,689,2021-07-30
3,2021-10-01,37814048,87,59824386,185,2021-05-28
4,2021-10-01,33955223,75,44727195,640,2020-07-15


In [101]:
#teste
#avaliar a formatação do mês atual

In [102]:
mes_atual

Timestamp('2021-10-01 00:00:00')

In [103]:
assingments_1 = assingments_process_portfolio[(assingments_process_portfolio.PortfolioID.isin(portifolios)) & (~assingments_process_portfolio.AgencyID.isin([716, 175,756,820])) & (assingments_process_portfolio.RefDate == mes_atual)]

In [104]:
assingments_1.columns = ["time", "debt_id", "portfolio_id", "customer_id", "agency_id", "assignment_date"]

In [105]:
assingments_1.head()

,time,debt_id,portfolio_id,customer_id,agency_id,assignment_date
0,2021-10-01,30109227,66,26954370,685,2020-02-01
1,2021-10-01,33897309,75,54904329,794,2021-07-30
2,2021-10-01,24913704,57,25676425,689,2021-07-30
3,2021-10-01,37814048,87,59824386,185,2021-05-28
4,2021-10-01,33955223,75,44727195,640,2020-07-15


In [106]:
val__assingments_1 = len(assingments_1)

In [107]:
# Inserindo segmento em assingments_1
assingments_1 = pd.merge(assingments_1, segment_portfolio, validate="many_to_one")

In [108]:
assert val__assingments_1 == len(assingments_1), \
            "assingments_1 mudou de tamanho ao inserir o segmento"

In [109]:
assingments_1.drop(["portfolio_id", "assignment_date"], axis=1, inplace=True)

In [110]:
assingments_1.head()

,time,debt_id,customer_id,agency_id,segment_id
0,2021-10-01,30109227,26954370,685,22
1,2021-10-01,30534504,49708804,686,22
2,2021-10-01,30851071,7822391,686,22
3,2021-10-01,30092299,43129621,680,22
4,2021-10-01,31262058,45783169,686,22


### Verifica para quais agências as dívidas foram distribuidas no mês anterior 

In [111]:
#teste
#avaliar a formatação do mês anterior

In [113]:
assingments_2 = assingments_process_portfolio[(assingments_process_portfolio.PortfolioID.isin(portifolios)) & (~assingments_process_portfolio.AgencyID.isin([716, 175,756,820])) & (assingments_process_portfolio.RefDate == mes_anterior)] 

In [114]:
assingments_2.columns = ["time", "debt_id", "portfolio_id", "customer_id", "agency_id", "assignment_date"]

In [115]:
val__assingments_2 = len(assingments_2)

In [116]:
# Inserindo segmento em assingments_2
assingments_2 = pd.merge(assingments_2, segment_portfolio, validate="many_to_one")

In [117]:
assert val__assingments_2 == len(assingments_2), \
            "assingments_2 mudou de tamanho ao inserir o segmento"

In [118]:
assingments_2.head()

,time,debt_id,portfolio_id,customer_id,agency_id,assignment_date,segment_id
0,2021-09-01,33212801,69,52692450,689,2021-08-31,17
1,2021-09-01,33067325,69,52771183,689,2021-04-29,17
2,2021-09-01,33252016,69,52756266,689,2020-01-14,17
3,2021-09-01,33118011,69,52652623,689,2021-02-27,17
4,2021-09-01,33101759,69,52806760,689,2021-07-30,17


In [119]:
assingments_2.drop(["portfolio_id", "assignment_date"], axis=1, inplace=True)

In [120]:
assingments = pd.concat([assingments_1, assingments_2])
assingments["time"] = pd.to_datetime(assingments["time"], format="%Y-%m-%d")

In [121]:
#teste
assingments = assingments.drop_duplicates(["time", "debt_id", "customer_id", "segment_id"])

In [122]:
count_assingments = assingments.groupby([
    "time", "debt_id"])["debt_id"].count()
assert (count_assingments == 1).all(), (
    "portfolio_id: Existe algum debtos com mais de um "
    "assingment")

In [123]:
assingments[assingments.debt_id == 12234496]

,time,debt_id,customer_id,agency_id,segment_id
12207590,2021-10-01,12234496,19535534,790,18
13946445,2021-09-01,12234496,19535534,790,18


In [124]:
del assingments_1, assingments_2, assingments_process_portfolio

#### Ajustando as dividas da FNX, a agencia 660 e 481 devem ser agrupadas para a distribuição

In [125]:
assingments["agency_id"] = assingments["agency_id"].replace({660: 481})

## Verifica as dívidas de forma distribuidas, quais estavam aptas para cobrança (não estavam em acordo ativo, ou judicializado)

#### a tabela database__debt__time_categorical é a debtcontacts para cada mês

In [126]:
#debtcontacts_columns = pd.read_parquet('s3://dlr-dev-bucket-refinedzone/pic/datamart/debtcontacts/', columns=["PortfolioID","DebtID", "CustomerID", "DebtStatus", "DebtCreationDate"])

In [127]:
#teste
debtcontacts_columns = pd.read_csv('s3://dlr-dev-bucket-refinedzone/pic/datamart/teste/database_debt_time_categorical_202201221101.csv')

In [128]:
#teste
debtcontacts_columns2 = pd.read_csv('s3://dlr-dev-bucket-refinedzone/pic/datamart/teste/database_debt_time_categorical_202201221204.csv')

In [129]:
debtcontacts_columns = pd.concat([debtcontacts_columns, debtcontacts_columns2])

In [132]:
#teste - analisar como esses dados vão vir
#debtcontacts_columns['DebtCreationDate'] = pd.to_datetime(debtcontacts_columns['DebtCreationDate']).dt.date

In [133]:
#teste - analisar como esses dados vão vir
#debtcontacts_columns['DebtCreationDate'] = debtcontacts_columns['DebtCreationDate'].apply(lambda dt: dt.replace(day=1))

In [136]:
debtcontacts_columns["time"] = pd.to_datetime(debtcontacts_columns["time"])

In [138]:
#debt_status_1 = debtcontacts_columns[(debtcontacts_columns.DebtStatus == 7000)  & (debtcontacts_columns.PortfolioID.isin(portifolios)) & (debtcontacts_columns.DebtCreationDate == mes_atual)]

In [140]:
debt_status_1 = debtcontacts_columns[(debtcontacts_columns.debt_status == 7000)  & (debtcontacts_columns.portfolio_id.isin(portifolios)) & (debtcontacts_columns.time == mes_atual)]

In [142]:
debt_status_1['debt_status'] = debt_status_1['debt_status'].astype('int')

/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':


In [143]:
#teste - analisar ordem das colunas
debt_status_1.columns = ["time", "portfolio_id", "debt_id", "customer_id", "debt_status"]

In [144]:
val__debt_status_1 = len(debt_status_1)

In [145]:
debt_status_1 = pd.merge(debt_status_1, segment_portfolio, validate="many_to_one")

In [146]:
assert val__debt_status_1 == len(debt_status_1), \
            "debt_status_1 mudou de tamanho ao inserir o segmento"

In [147]:
debt_status_1.drop(["portfolio_id"], axis=1, inplace=True)

#### Pegando as dívidas distribuidas do mês anterior

In [150]:
#debt_status_2 = debtcontacts_columns[(debtcontacts_columns.DebtStatus == 7000)  & (debtcontacts_columns.PortfolioID.isin(portifolios)) & (debtcontacts_columns.DebtCreationDate == mes_anterior)]

In [151]:
debt_status_2 = debtcontacts_columns[(debtcontacts_columns.debt_status == 7000)  & (debtcontacts_columns.portfolio_id.isin(portifolios)) & (debtcontacts_columns.time == mes_anterior)]

In [152]:
debt_status_2['debt_status'] = debt_status_2['debt_status'].astype('int')

/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':


In [153]:
debt_status_2.columns = [ "time", "portfolio_id", "debt_id","customer_id", "debt_status"]

In [154]:
val__debt_status_2 = len(debt_status_2)

In [155]:
# Inserindo segmento em debt_status_2
debt_status_2 = pd.merge(debt_status_2, segment_portfolio, validate="many_to_one")

In [156]:
assert val__debt_status_2 == len(debt_status_2), \
            "debt_status_2 mudou de tamanho ao inserir o segmento"

In [157]:
debt_status_2.drop(["portfolio_id"], axis=1, inplace=True)

In [158]:
debt_status = pd.concat([debt_status_1, debt_status_2])
debt_status["time"] = pd.to_datetime(debt_status["time"], format="%Y%m")

In [159]:
debt_status["customer_id"] = debt_status["customer_id"].astype(int)

In [160]:
del debt_status_1, debt_status_2, debtcontacts_columns

In [163]:
#######################################
# Associando o status de distribuição #
assingments_status = assingments.merge(debt_status, how="outer", validate="one_to_one") 

In [166]:
del debt_status, assingments

In [167]:
assingments_status['debt_status'] = assingments_status['debt_status'].fillna(0)

In [168]:
assingments_status['debt_status'] = assingments_status['debt_status'].astype('int')

In [169]:
assingments_status['debt_status'] = assingments_status['debt_status'].astype('str')

####  Ajustando o status quando não tem status

In [170]:
assingments_status["debt_status"] = assingments_status["debt_status"].replace(['0'], "sem_status")

#### Identifica as dívidas que estão ajuizadas

In [172]:
index_ajuizado = assingments_status["customer_id"].isin(customer_ajuizado["customer_id"].to_list())
assingments_status.loc[index_ajuizado, "debt_status"] = "ajuizado"

In [173]:
# index_na_month = assingments_status["months_in_agency"].isna()
# assingments_status = assingments_status[~index_na_month].copy()

#### Informações da dívida 

In [174]:
debtcontacts = pd.read_parquet('s3://dlr-dev-bucket-refinedzone/pic/datamart/debtcontacts/', columns=["PortfolioID", "Portfolio", "DebtID", "Products", "FirstDefaultDate", "OriginalFirstDefaultBalance"])

In [175]:
debt_info = debtcontacts[debtcontacts.PortfolioID.isin(portifolios)]

In [176]:
del debtcontacts

In [177]:
debt_info = debt_info.rename(columns={"PortfolioID":"portfolio_id", "Portfolio":"portfolio", "DebtID": "debt_id", "Products": "products", "OriginalFirstDefaultBalance":"reference_balance", "FirstDefaultDate":"first_default_date"})

#### Fazendo o cálculo de idade da dívida para meses completos 

In [178]:
debt_info["first_default_date"] = debt_info["first_default_date"].dt.floor("d") - pd.offsets.MonthBegin(1)

In [179]:
debt_info.head()

,portfolio_id,portfolio,debt_id,products,first_default_date,reference_balance
0,79.0,Santander,12230411.0,Capital de Giro,1998-06-01,706433.00
1,79.0,Santander,12230412.0,Cartão de Crédito,2011-03-01,655.62
2,79.0,Santander,12230413.0,Cartão de Crédito,2012-02-01,694.36
3,79.0,Santander,12230414.0,Cartão de Crédito,2010-09-01,10.70
4,79.0,Santander,12230416.0,Cartão de Crédito,2010-12-01,248.11


In [180]:
len_assingments = len(assingments_status)

In [181]:
assingments_status = debt_info.merge(assingments_status, validate="one_to_many")

In [182]:
assert len_assingments == len(assingments_status), ("Algumas dívidas distribuidas não tem debt info.")

### Query para cash novo deve pegar os dados do mês de distribuição e do mês anterior

In [183]:
#teste
mes_anterior = pd.to_datetime('2021-09-01', format="%Y-%m-%d")
mes_atual = pd.to_datetime('2021-10-01', format="%Y-%m-%d")

In [185]:
payments["RefDate"] = pd.to_datetime(payments["RefDate"], format='%Y%m')

In [186]:
cash_novo = payments[(payments.PortfolioID.isin(portifolios)) & (payments.CashType == 'cash_novo') & (payments.RefDate.isin([mes_atual, mes_anterior]))][["RefDate", "PortfolioID", "DebtID", "CustomerID", "AgencyID", "CollectionAmount" ]]

In [187]:
cash_novo = cash_novo.rename(columns={"RefDate":"time", "PortfolioID":"portfolio_id", "DebtID":"debt_id", "CustomerID":"customer_id", "AgencyID":"agency_id", "CollectionAmount":"cash_novo"})

#### Ajustando as dividas da FNX, a agencia 660 e 481 devem ser agrupadas para a distribuição

In [189]:
cash_novo["agency_id"] = cash_novo["agency_id"].replace({660: 481})

In [190]:
cash_novo["cash_novo"] = pd.to_numeric(cash_novo["cash_novo"])

In [192]:
cash_novo = cash_novo.groupby(["time", "portfolio_id", "debt_id", "agency_id", "customer_id"], as_index=False, observed=True)["cash_novo"].sum()

In [196]:
assingments_status = assingments_status.merge(cash_novo,  how="left", validate="one_to_one")

In [197]:
assingments_status["cash_novo"] = assingments_status["cash_novo"].fillna(value=0)

### Calculando os cortes 

In [199]:
delta_days = assingments_status["time"] - assingments_status["first_default_date"]
debt_age_years = (delta_days / np.timedelta64(1, "M")).round() / 12

In [200]:
debt_age_years_cut = pd.cut(debt_age_years, bins=cuts_age, right=True, labels=cuts_age[:-1])

In [201]:
# assert not debt_age_years_cut.isna().any(), (
#             "Algum corte do debt age está sendo feito errado.")

In [202]:
assingments_status["debt_age_years_cut"] = debt_age_years_cut

In [203]:
teste
assingments_status[assingments_status["debt_age_years_cut"].isna()]

,portfolio_id,portfolio,debt_id,products,first_default_date,reference_balance,time,customer_id,agency_id,segment_id,debt_status,cash_novo,debt_age_years_cut
2256397,82.0,CitiBank3,14070125.0,None,NaT,None,2021-10-01,26937568,481.0,11,7000,0.0,NaN
2256398,82.0,CitiBank3,14070125.0,None,NaT,None,2021-09-01,26937568,481.0,11,7000,0.0,NaN


In [204]:
# assert not assingments_status.isna().any().any(),\
#             "Sobrou algum NA na base de dados."

In [205]:
assingments_status = assingments_status[~assingments_status["debt_age_years_cut"].isna()]

In [206]:
##############################################
# Ponderando por 60/40 #poderacao__cash_novo #
# Para o calculo da eficiencia considera todo o valor de cash novo
poderacao__cash_novo = assingments_status.groupby(["time", "agency_id", "segment_id", "products","debt_age_years_cut"], as_index=False, observed=True)["cash_novo"].sum()

In [207]:
cash_mes_atual = poderacao__cash_novo[poderacao__cash_novo["time"] == pd.to_datetime(mes_atual, format="%Y-%m-%d")].rename(columns={"cash_novo": "cash_novo__mes_atual"})

In [208]:
del cash_mes_atual["time"]

In [209]:
cash_mes_anterior = poderacao__cash_novo[poderacao__cash_novo["time"] == pd.to_datetime(mes_anterior, format="%Y-%m-%d")].rename(columns={"cash_novo": "cash_novo__mes_anterior"})

In [210]:
del cash_mes_anterior["time"]

In [211]:
merged_cash = cash_mes_anterior.merge(cash_mes_atual, how="outer")

In [212]:
assingment_status_corrigido = assingments_status.convert_dtypes()

In [213]:
assingments_status['debt_status'] = assingment_status_corrigido["debt_status"].astype('str')

### Para o calculo da eficiencia considerar apenas o valor de face livre (dívidas disponíveis) que está disponível em 7000 e que não está ajuizado


In [214]:
index_dividas_livres = assingments_status["debt_status"] == '7000'

In [215]:
poderacao__reference_balance = assingments_status[index_dividas_livres].groupby(["time", "agency_id", "segment_id", "products", "debt_age_years_cut"],as_index=False, observed=True)["reference_balance"].sum()

In [216]:
index_actual = poderacao__reference_balance["time"] == pd.to_datetime(mes_atual, format="%Y-%m-%d")

In [217]:
face_mes_atual = poderacao__reference_balance[index_actual].rename(columns={"reference_balance": "face__total_cobranca__mes_atual"})

In [218]:
del face_mes_atual["time"]

In [219]:
index_previous = poderacao__reference_balance["time"] == pd.to_datetime(mes_anterior, format="%Y-%m-%d")
face_mes_anterior = poderacao__reference_balance[index_previous].rename(columns={"reference_balance": "face__total_cobranca__mes_anterior"})

In [220]:
del face_mes_anterior["time"]

In [221]:
merged_face = face_mes_anterior.merge(face_mes_atual, how="outer")

In [222]:
# Unindo as duas bases
# poderacao = poderacao__cash_novo.merge(poderacao__reference_balance, validate="one_to_one",how="outer")
# poderacao["cash_novo"].fillna(value=0, inplace=True)

In [223]:
# validando ponderacao
# index_na_reference_balance = poderacao["reference_balance"].isna()
# if index_na_reference_balance.any():
#     poderacao["reference_balance"].fillna(value=0, inplace=True)
#     poderacao = poderacao[poderacao["reference_balance"] != 0].copy()

 Taiane, não entendi a celula abaixo, é pra executar ou não?

In [224]:
# poderacao.reset_index(inplace=True)

In [225]:
efficiencia = merged_cash.merge(merged_face, how="outer")#.merge(poderacao)

In [226]:
#new_assignments_template são as dívidas que estão disponíveis no momento da otimização
assingments_7000 = new_assignments_template

In [227]:
len(new_assignments_template)

1758427

In [228]:
new_assignments_template.head()

,assignment_time,portfolio_id,debt_id,customer_id,agency_id,binding_id
818080,2021-02-27,46.0,14070137,26969668.0,481,14154373.0
1621317,2021-06-02,46.0,14070139,26965502.0,640,14369179.0
649186,2021-05-28,46.0,14070141,26988066.0,691,14152418.0
1672105,2021-06-29,46.0,14070144,26965416.0,635,14163849.0
838719,2021-08-31,46.0,14070157,26976693.0,481,14167591.0


In [229]:
del new_assignments_template

In [230]:
assingments_7000["agency_id"] = assingments_7000["agency_id"].replace({660: 481})

In [231]:
# Validando se assingments_7000 vem informações
if(len(assingments_7000) < 1):
    print("O portflio {} não tem assingments_7000")
    base_otimizacao = pd.DataFrame([])
    debts_livres: pd.DataFrame([])

In [232]:
###################################################################
# Colocando uma margem de 10 dias para o caso de dividas que são
# distribuidas até o dia 10 ainda são consideradas com parte do mes
# as que forem posteriores a 10 dias passam a ser consideradas como
# distribuidas no próximo mês
assingments_7000["reference_month"] = (assingments_7000["assignment_time"] - pd.offsets.Day(11)) + pd.offsets.MonthBegin(1)

In [233]:
assingments_7000.head()

,assignment_time,portfolio_id,debt_id,customer_id,agency_id,binding_id,reference_month
818080,2021-02-27,46.0,14070137,26969668.0,481,14154373.0,2021-03-01
1621317,2021-06-02,46.0,14070139,26965502.0,640,14369179.0,2021-06-01
649186,2021-05-28,46.0,14070141,26988066.0,691,14152418.0,2021-06-01
1672105,2021-06-29,46.0,14070144,26965416.0,635,14163849.0,2021-07-01
838719,2021-08-31,46.0,14070157,26976693.0,481,14167591.0,2021-09-01


In [234]:
optimization_month = pd.to_datetime('2021-11-01')

In [235]:
assingments_7000["months_in_agency_opt_month"] = ((optimization_month - assingments_7000["reference_month"])/np.timedelta64(1, "M")).round()

In [236]:
# O prazo para a distribuição das dívidas é de 3 meses no escritório
# após 4 meses as dívidas são consideradas disponíveis para a
# redistribuição
assingments_7000["debt_status"] = "decurso de prazo"

In [237]:
assingments_7000.loc[ assingments_7000["months_in_agency_opt_month"] <= 3, "debt_status"] = "menor 4M"

In [238]:
# Marcando as dívidas que estão ajuizadas para que não estejam
# disponíveis para a redistribuição
index_ajuizado = assingments_7000["customer_id"].isin(customer_ajuizado["customer_id"])
assingments_7000.loc[index_ajuizado, "debt_status"] = "ajuizado"

In [239]:
# Verificando quais dividas não possuem bindingid_null
# elas não podem ser distribuidos
index_bindingid_null = assingments_7000["debt_id"].isin(debt_bindingid_null["debt_id"].to_list())
assingments_7000.loc[index_bindingid_null, "debt_status"] = "bindingid_null"

In [241]:
# Criando segment id na base de assigments
assingments_7000 = assingments_7000.merge(segment_portfolio, on="portfolio_id", validate="many_to_one")

In [242]:
val__segment = assingments_7000[["portfolio_id", "segment_id"]].drop_duplicates().groupby("portfolio_id")["segment_id"].count()
assert (val__segment.values == 1).all(), \
    "Há portfolios vinculados a mais de um segmento"

In [243]:
val__restriction = len(AssignmentsOptimizationRestrictive[AssignmentsOptimizationRestrictive.DistributionDate == reference_date])

In [244]:
# Verificando quais escritórios são permitidos de cobrar as dívidas
# caso o escritório não esteja is_enabled todos as dívidas são
# colocadas para distribuição
if val__restriction == 0:
    # Se não foram carregados segmentos, considerar todos como
    # disponíveis para receber dívidas
    assingments_7000["is_enabled"] = True
    assingments_7000["enforced_flag"] = False
else:
    # Caso contrário consiera os segmentos que não foram colocados
    # como não disponíveis
    assingments_7000 = assingments_7000.merge(
        segment_restriction[[
            'segment_id', 'agency_id', 'is_enabled', 'enforced_flag']],
        how="left")
    assingments_7000["is_enabled"].fillna(value=False, inplace=True)
    assingments_7000["enforced_flag"].fillna(value=False, inplace=True)

In [245]:
index_not_enabled = ( ~assingments_7000["is_enabled"] | assingments_7000["enforced_flag"])

In [246]:
# Caso um escritório seja descadastrados e sua dívida não esteja
# ajuizada essa divida deve ser distribuida independente do decurso
# de prazo
index_not_ajuizado = ~assingments_7000["debt_status"].isin(["ajuizado", "bindingid_null"])
assingments_7000.loc[index_not_enabled & index_not_ajuizado,"debt_status"] = "descadastrado"

In [247]:
val_n = len(assingments_7000)

In [252]:
val_n = len(assingments_7000)

In [253]:
assingments_7000 = pd.merge(assingments_7000, debt_info, how="left", validate="one_to_one")

In [255]:
assert len(assingments_7000) == val_n,\
    "len(assingments_7000) == val_n ."

In [256]:
delta_days = optimization_month - assingments_7000["first_default_date"]
debt_age_years = (delta_days / np.timedelta64(1, "M")).round() / 12

In [257]:
debt_age_years_cut = pd.cut(debt_age_years, bins=cuts_age, right=True, labels=cuts_age[:-1])

In [258]:
assert not debt_age_years_cut.isna().any(), (
    "Algum corte do debt age está sendo feito errado [%s].")

In [259]:
assingments_7000["debt_age_years_cut"] = debt_age_years_cut

In [260]:
index_debts_cobranca = ~assingments_7000["debt_status"].isin(["ajuizado", "bindingid_null"])

In [261]:
assingments_em_cobranca = assingments_7000.loc[index_debts_cobranca, [
        "segment_id", "debt_id", "binding_id", "customer_id",
        "agency_id", "debt_age_years_cut", "products",
        "debt_status", "assignment_time", "reference_month",
        "months_in_agency_opt_month", "reference_balance"]]

In [263]:
assert not assingments_em_cobranca["agency_id"].isin([716, 175, 756, 820]).any(), (
    "assingments_em_cobranca: Existem dividas das "
    "agencias 716, 175, 756, 820 (Vellum...)")

### Criando a base de otimização 

In [264]:
grouped_assingments_7000 = assingments_7000.groupby([
    "segment_id", "products", "debt_age_years_cut",
    "debt_status", "agency_id", "is_enabled", "enforced_flag"],
    observed=True)["reference_balance"].sum()

In [265]:
grouped_assingments_7000 = grouped_assingments_7000.unstack("debt_status")
grouped_assingments_7000.fillna(value=0, inplace=True)

In [266]:
# Soma as dívidas que não estão ajuizadas como total de cobrança
grouped_assingments_7000["total_cobranca"] = grouped_assingments_7000.drop(["ajuizado", "bindingid_null"], axis=1, errors="ignore").sum(axis=1)

In [267]:
livre_columns = [x for x in ["decurso de prazo", "descadastrado"] if x in grouped_assingments_7000.columns]
grouped_assingments_7000["face_livre"] = grouped_assingments_7000[livre_columns].sum(axis=1)
grouped_assingments_7000.reset_index(inplace=True)

In [268]:
# Unindo os debtos livres aos calculos de performance
base_otimizacao = efficiencia.merge(grouped_assingments_7000, how="right")

In [269]:
assert not base_otimizacao["agency_id"].isin([
    716, 175, 756, 820]).any(), (
        "base_otimizacao: Existem dividas das agencias 716, 175, "
        "756, 820 (Vellum...)")

In [270]:
all_agg_results = base_otimizacao
del base_otimizacao 

In [271]:
pd_free_debts = assingments_em_cobranca
del assingments_em_cobranca

In [272]:
all_agg_results["time"] = optimization_month
all_agg_results["agency_id"] = pd.to_numeric(all_agg_results["agency_id"])

In [273]:
# Agregando resultados por segmento
if "descadastrado" not in all_agg_results.columns:
    all_agg_results["descadastrado"] = 0

In [274]:
all_agg_results= all_agg_results.convert_dtypes()

In [275]:
all_agg_results["face__total_cobranca__mes_anterior"] = pd.to_numeric(all_agg_results["face__total_cobranca__mes_anterior"])

In [276]:
all_agg_results["face__total_cobranca__mes_atual"] = pd.to_numeric(all_agg_results["face__total_cobranca__mes_atual"])

In [277]:
all_agg_results["ajuizado"] = pd.to_numeric(all_agg_results["ajuizado"])

In [278]:
all_agg_results["decurso de prazo"] = pd.to_numeric(all_agg_results["decurso de prazo"])

In [279]:
all_agg_results.head()

,agency_id,segment_id,products,debt_age_years_cut,cash_novo__mes_anterior,cash_novo__mes_atual,face__total_cobranca__mes_anterior,face__total_cobranca__mes_atual,is_enabled,enforced_flag,ajuizado,decurso de prazo,descadastrado,menor 4M,total_cobranca,face_livre,time
0,185,1,Cartão de Crédito,0,6200.09,486.08,36921617.88,37539891.09,True,False,18828.40,1821746.96,0,3581519.61,5403266.57,1821746.96,2021-11-01
1,571,1,Cartão de Crédito,0,2274.03,603.50,8408910.83,8560811.80,True,False,6692.95,445822.47,0,597616.74,1043439.21,445822.47,2021-11-01
2,640,1,Cartão de Crédito,0,52.88,0.00,5235560.94,5608211.71,True,False,2912.08,654449.66,0,0,654449.66,654449.66,2021-11-01
3,652,1,Cartão de Crédito,0,0.00,0.00,NaN,3523.73,False,False,0.00,0.00,2248.88,0,2248.88,2248.88,2021-11-01
4,669,1,Cartão de Crédito,0,0.00,0.00,814.29,26056.18,False,False,0.00,0.00,26056.18,0,26056.18,26056.18,2021-11-01


In [280]:
aggregation_data = all_agg_results.groupby([
    "time", "is_enabled", "enforced_flag", "agency_id", "segment_id",
    "products", "debt_age_years_cut"], as_index=False, observed=True)[[
        "cash_novo__mes_anterior",
        "cash_novo__mes_atual",
        "face__total_cobranca__mes_anterior",
        "face__total_cobranca__mes_atual",
        "ajuizado",
        "decurso de prazo",
        "descadastrado",
        "menor 4M",
        "total_cobranca",
        "face_livre"]].sum()

In [281]:
all_agg_results.head()

,agency_id,segment_id,products,debt_age_years_cut,cash_novo__mes_anterior,cash_novo__mes_atual,face__total_cobranca__mes_anterior,face__total_cobranca__mes_atual,is_enabled,enforced_flag,ajuizado,decurso de prazo,descadastrado,menor 4M,total_cobranca,face_livre,time
0,185,1,Cartão de Crédito,0,6200.09,486.08,36921617.88,37539891.09,True,False,18828.40,1821746.96,0,3581519.61,5403266.57,1821746.96,2021-11-01
1,571,1,Cartão de Crédito,0,2274.03,603.50,8408910.83,8560811.80,True,False,6692.95,445822.47,0,597616.74,1043439.21,445822.47,2021-11-01
2,640,1,Cartão de Crédito,0,52.88,0.00,5235560.94,5608211.71,True,False,2912.08,654449.66,0,0,654449.66,654449.66,2021-11-01
3,652,1,Cartão de Crédito,0,0.00,0.00,NaN,3523.73,False,False,0.00,0.00,2248.88,0,2248.88,2248.88,2021-11-01
4,669,1,Cartão de Crédito,0,0.00,0.00,814.29,26056.18,False,False,0.00,0.00,26056.18,0,26056.18,26056.18,2021-11-01


In [283]:
all_agg_results["descadastrado"] = all_agg_results["descadastrado"].astype(int)

In [284]:
all_agg_results["menor 4M"] = all_agg_results["menor 4M"].astype(float)

In [285]:
aggregation_data = all_agg_results.groupby([
    "time", "is_enabled", "enforced_flag", "agency_id", "segment_id",
    "products", "debt_age_years_cut"], as_index=False, observed=True)[[
        "cash_novo__mes_anterior",
        "cash_novo__mes_atual",
        "face__total_cobranca__mes_anterior",
        "face__total_cobranca__mes_atual",
        "ajuizado",
        "menor 4M",
        "decurso de prazo","descadastrado",
        "total_cobranca",
        "face_livre"]].sum()

In [286]:
aggregation_data["bloqueado"] = aggregation_data["total_cobranca"] - aggregation_data["face_livre"]

In [287]:
# Fazendo o cálculo da eficiência ponderada
aggregation_data["cash_novo__ponderado"] = aggregation_data["cash_novo__mes_anterior"] * 0.4 + aggregation_data["cash_novo__mes_atual"] * 0.6

In [288]:
aggregation_data["face__total_cobranca__ponderado"] = aggregation_data["face__total_cobranca__mes_anterior"] * 0.4 + aggregation_data["face__total_cobranca__mes_atual"] * 0.6

In [289]:
aggregation_data["eficiencia__60_40"] = aggregation_data["cash_novo__ponderado"] / aggregation_data["face__total_cobranca__ponderado"]

In [291]:
# Adicinando o nome da agência a base de dados para facilitar as
# validações
agency_to_merge = agency.rename(columns={"name": "agency"})[["agency_id", "agency"]]
n_val = len(aggregation_data)
aggregation_data = agency_to_merge.merge(aggregation_data[[
    "time", "agency_id", "is_enabled", "enforced_flag", "segment_id",
    "products", "debt_age_years_cut", "cash_novo__mes_anterior",
    "cash_novo__mes_atual", "cash_novo__ponderado",
    "face__total_cobranca__mes_anterior",
    "face__total_cobranca__mes_atual",
    "face__total_cobranca__ponderado",
    "eficiencia__60_40",
    "ajuizado",
    "menor 4M",
    "decurso de prazo",
    "descadastrado",
    "bloqueado",
    "face_livre",
    "total_cobranca"]], validate="one_to_many")

In [292]:
assert len(aggregation_data) == n_val, \
    "Algumas agências não estão no de-para [agency_to_merge]"

In [293]:
results_val = aggregation_data.groupby(["time", "agency_id", "segment_id", "products","debt_age_years_cut"], observed=True)["cash_novo__ponderado"].count()

In [294]:
assert (1 == results_val).all(),\
    "Existem valores duplicados"

#### Formatando os tipos das colunas

In [344]:
pd_free_debts["binding_id"] = pd_free_debts["binding_id"].astype("int64")
pd_free_debts["customer_id"] = pd_free_debts["customer_id"].astype("int64")
pd_free_debts["reference_balance"] = pd_free_debts["reference_balance"].astype("float64")

In [347]:
aggregation_data["descadastrado"] = aggregation_data["descadastrado"].astype("float64") 

In [350]:
pd_free_debts = pd_free_debts.rename(columns={'products': 'product'})
aggregation_data = aggregation_data.rename(columns={'products': 'product'})

#### Salvar no s3 como parquet

In [364]:
now = datetime.datetime.now().isoformat()

In [365]:
month_optimization = "11"

#### Salvando a base de debtos livres

In [366]:
s3_file_free_debts = 'models/data_prep/'+yearMonth+'-'+month_optimization+'/free_debts__'+now+'.parquet'
wr.s3.to_parquet(pd_free_debts,path='s3://dlr-dev-bucket-refinedzone/models/data_prep/'+yearMonth+'-'+month_optimization+'/free_debts__'+now+'.parquet')    

{'paths': ['s3://dlr-dev-bucket-refinedzone/models/data_prep/2021-11/free_debts__2022-01-24T19:59:39.324957.parquet'],
 'partitions_values': {}}

#### Salvando as bases de otimização

In [367]:
s3_file_optimization_db = 'models/data_prep/'+yearMonth+'-'+month_optimization+'/otimization_db__'+now+'.parquet'
wr.s3.to_parquet(aggregation_data,path='s3://dlr-dev-bucket-refinedzone/models/data_prep/'+yearMonth+'-'+month_optimization+'/otimization_db__'+now+'.parquet')

{'paths': ['s3://dlr-dev-bucket-refinedzone/models/data_prep/2021-11/otimization_db__2022-01-24T19:59:39.324957.parquet'],
 'partitions_values': {}}

#### Salvando o metadata

In [368]:
# Salvando a metadata do run
s3_file_metadata = 'models/data_prep/'+yearMonth+'-'+month_optimization+'/metadata__'+now+'.json'
last_run_dict = {
    "run_time": now,
    "files": {
        "otimization_db": s3_file_optimization_db,
        "free_debts_db": s3_file_free_debts,
        "metadata": s3_file_metadata,
    },
    "optimization_month": optimization_month.isoformat(),
    "mes_atual": mes_atual.isoformat(),
    "mes_anterior": mes_anterior.isoformat(),
    "cuts_age": mes_anterior.isoformat(),
    "portfolio_ids__enabled": portfolio_ids__enabled[
        "PortfolioID"].tolist(),
}

In [369]:
run_metadata = json.dumps(last_run_dict, indent=2).encode("utf-8")

In [370]:
s3 = boto3.resource('s3')
s3object = s3.Object('dlr-dev-bucket-refinedzone', 'models/data_prep/'+yearMonth+'-'+month_optimization+'/metadata__'+now+'.json')


In [371]:
s3object.put(
    Body=(bytes(run_metadata))
)

{'ResponseMetadata': {'RequestId': 'R6MTKDWJFA121Q5Y',
  'HostId': 'XGFCN2LCKPdloHVADoLM/up5DFrPo1tBDa+kskB6Q08OexL3s4NW8eo/BimbGbBVVwh1eQGl6vE=',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amz-id-2': 'XGFCN2LCKPdloHVADoLM/up5DFrPo1tBDa+kskB6Q08OexL3s4NW8eo/BimbGbBVVwh1eQGl6vE=',
   'x-amz-request-id': 'R6MTKDWJFA121Q5Y',
   'date': 'Mon, 24 Jan 2022 19:59:43 GMT',
   'x-amz-version-id': 'WhOwzU4oGPESJWg36WwUCxs2VuJlWqc.',
   'x-amz-server-side-encryption': 'aws:kms',
   'x-amz-server-side-encryption-aws-kms-key-id': 'arn:aws:kms:us-east-2:642344871054:key/b50764cd-f3a7-420e-92a4-a259917163ab',
   'etag': '"cf6f80ddb9ea23c8a235ab2d5d31594a"',
   'server': 'AmazonS3',
   'content-length': '0'},
  'RetryAttempts': 0},
 'ETag': '"cf6f80ddb9ea23c8a235ab2d5d31594a"',
 'ServerSideEncryption': 'aws:kms',
 'VersionId': 'WhOwzU4oGPESJWg36WwUCxs2VuJlWqc.',
 'SSEKMSKeyId': 'arn:aws:kms:us-east-2:642344871054:key/b50764cd-f3a7-420e-92a4-a259917163ab'}

In [372]:
s3object_last = s3.Object('dlr-dev-bucket-refinedzone', 'models/data_prep/'+yearMonth+'-'+month_optimization+'/metadata__last_run.json')


In [374]:
s3object_last.put(
    Body=(bytes(run_metadata))
)

{'ResponseMetadata': {'RequestId': 'R6MQYN4P0BK74TXV',
  'HostId': '7U9I708gkHwBr53leHde/TWDaP2vw7hszSLlns9Xf4rLBawA7DKaHXreTSWCbH/VJSTPq8a2qVI=',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amz-id-2': '7U9I708gkHwBr53leHde/TWDaP2vw7hszSLlns9Xf4rLBawA7DKaHXreTSWCbH/VJSTPq8a2qVI=',
   'x-amz-request-id': 'R6MQYN4P0BK74TXV',
   'date': 'Mon, 24 Jan 2022 19:59:43 GMT',
   'x-amz-version-id': 'MFGOAcCrS3cUInxETfxfV1N1uzpoueyC',
   'x-amz-server-side-encryption': 'aws:kms',
   'x-amz-server-side-encryption-aws-kms-key-id': 'arn:aws:kms:us-east-2:642344871054:key/b50764cd-f3a7-420e-92a4-a259917163ab',
   'etag': '"d2df80d58185ab66cc92eeb413cb1f9a"',
   'server': 'AmazonS3',
   'content-length': '0'},
  'RetryAttempts': 0},
 'ETag': '"d2df80d58185ab66cc92eeb413cb1f9a"',
 'ServerSideEncryption': 'aws:kms',
 'VersionId': 'MFGOAcCrS3cUInxETfxfV1N1uzpoueyC',
 'SSEKMSKeyId': 'arn:aws:kms:us-east-2:642344871054:key/b50764cd-f3a7-420e-92a4-a259917163ab'}